In this code, I decided to try the DQN method on the same "taxi-v3", through the policy based method suing Boltzmann policy for the rules.

In [1]:
%pip install gym
%pip install tensorflow==2.4.1
%pip install keras-rl2
#installing gymnasium, tensorflow and keras

     -------------------------------------- 721.7/721.7 kB 4.2 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for gym: filename=gym-0.26.2-py3-none-any.whl size=827629 sha256=7ada41539e996d37e9561e01893695a8fa19390992236d3acf817a5092ab3898
  Stored in directory: c:\users\kushi arun kumar\appdata\local\pip\cache\wheels\e0\d1\88\c3c712101dac979e0bf77cdd9c89bc650ad5f41c48d8d6f167
Successfully built gym
Note: you may need to restart the kernel to use updated packages.
     -------------------------------------- 370.7/370.7 MB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ----------------------------

In [ ]:
import numpy as np
import gym
import random

#importing necessary libraries and modules - here there is, numpy for numerical calculations, gym to create the environment 

In [50]:
env_name="Taxi-v3"
env=gym.make(env_name, render_mode="ansi")

#creating the environment to interact with through function "gym.make" from the gymnasium model

In [72]:
env.reset(seed=123)

#Resetting the environment to its initial state and returning the initial observation
#Setting the random seed to 123 makes it so that the envoirponment behaves consistewntly across different systems which allows for uniformity in running the program
#Resets the environment env to its initial state.


(341, {'prob': 1.0, 'action_mask': array([1, 1, 0, 1, 0, 0], dtype=int8)})

In [73]:
actions=env.action_space.n #retrieving the total number of actions 
states=env.observation_space.n #retrieving the total number of different observational states
print("Number of actions: ", actions) #comes out to be 6
print("Number of states: ", states) #comes out to be 500

Number of actions:  6
Number of states:  500


In [74]:

episodes=10
for episode in range(1, episodes+1):
    state = env.reset() if isinstance(env.reset(), int) else env.reset()[0]
    done=False #if episode is not completed
    score=0 #set score to 0

    while not done:
        env.render()
        action=random.choice([0,1])
        n_state, reward, done, truncated, info=env.step(action)
        score+=reward #update the scores according to the rewards
    print("Episode:{} Score:{}".format(episode,score))

KeyboardInterrupt: 

In [127]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Activation, Embedding, Reshape, Input
from tensorflow.keras.optimizers import Adam

#importing few more necessary libraries and modules, tensorflow for building and training the neural networks  

In [128]:
env.reset() #resetting the env 
env.step(env.action_space.sample())[0] #this code is used to access the first element of the tuple returned by the stap metjod - which is the new state of the environment after the action has been executed

302

In [1]:
#function to build the model, only embedding
def build_model(states, actions): #
    '''model = Sequential()
    model.add(Dense(24, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))  # Output layer with one value per action
    model.compile(optimizer=Adam(lr=0.001), loss='mse')'''
    model = Sequential() #making the sequential model
    model.add(Embedding(500, 6, input_length=1)) #adding an embedding layer to represent the 500 states with 6 dimensions
    model.add(Reshape((6,))) #adding a reshape layer to flatten the output from the embedding layer.
    return model

In [172]:
model=build_model(states,actions) #rceates a new neural network that we have sccording to the states and actions

In [173]:
model.summary() #provides a summary of the neural network model we have created

Model: "sequential_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 1, 6)              3000      
_________________________________________________________________
reshape_10 (Reshape)         (None, 6)                 0         
Total params: 3,000
Trainable params: 3,000
Non-trainable params: 0
_________________________________________________________________


In [174]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory
#the dqn library helps by implementing the DQN algorithm for learning Q-values
#the BoltzmannQPolicy will help by providing a probabilistic action selection strategy based on Q-values, kind of like the epsgreedy strategy
#and finally the manages the storage of experiences for training the agent

In [133]:
%pip install gym[toy_text]
#I installed this additional dependency as to make sure that there is access to the additional text-based environments provided by Gym

^C
Note: you may need to restart the kernel to use updated packages.


In [187]:
def create_dqn_agent(env): #creating and configconfiguring a DQN agent
    model = build_model(states, actions) #calling build_model function
    policy=BoltzmannQPolicy() #defining the policy 
    memory=SequentialMemory(limit=50000, window_length=1) #used to store the experiences in terms of state, action, etc; limit is the maximum number of experiences to store in memory; window_length=1 means that each experience consists of a single state-action-reward-next state tuple else there will be multiple states
    dqn=DQNAgent(model=model, memory=memory, policy=policy, nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2) 

    return dqn

In [188]:
dqn = create_dqn_agent(env) #initializing the dqn agent in the environment



In [189]:
dqn.compile(Adam(learning_rate=0.001), metrics=['mae']) #this line of code sets the DQN agent for training
#Adam optimizer is an adaptive learning rate optimization algorithm and we are setting the learning rate to 0.001 here
#we are also setting the metrics for mean abosolute error

In [190]:
dqn.fit(env, nb_steps=5000, visualize=False, verbose=1) #we are training the DQN agent on the specified environment for 5,000 steps, without visualizing the training process


Training for 5000 steps ...
Interval 1 (0 steps performed)


ValueError: Error when checking input: expected embedding_13_input to have 2 dimensions, but got array with shape (1, 1, 2)

I faced multiple errors with this method of RL:
1. The first being that installing all tensorflow, keras and keras-rl gave an error while importing the libraries, hence I had to uninstall keras and keep keras-rl and tensorflow
2. This issue, I was not able to fix, the expected input shape was input data with 2 dimensions but the actual input shape had 3 dimensions with the shape (1, 1, 2), which suggests that it has an extra dimension. Due to this the command for dqn.fit did not work and I could'nt proceed with the same despite trying to solve the issue manier times. 

Hence this is all I could pull off in the short duration of our submission tasks.